In [1]:
#Load libraries
import pandas as pd
import numpy as np
import seaborn as sns
import re
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import text
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.linear_model import LinearRegression
%matplotlib inline

In [2]:
#Import datasets for train / test
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
#Create new dummy column to indicate orig dataframe
train['Dataset'] = 'Train'
test['Dataset'] = 'Test'

In [4]:
#Combine into new DataFrame 'Total'
total = train.append(test, ignore_index=True, sort=False)

In [5]:
#Check the head of new dataframe
total.head()

,id,belongs_to_collection,budget,genres,homepage,imdb_id,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,Keywords,cast,crew,revenue,Dataset
0,1,"[{'id': 313576, 'name': 'Hot Tub Time Machine ...",14000000,"[{'id': 35, 'name': 'Comedy'}]",NaN,tt2637294,en,Hot Tub Time Machine 2,"When Lou, who has become the ""father of the In...",6.575393,...,93.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The Laws of Space and Time are About to be Vio...,Hot Tub Time Machine 2,"[{'id': 4379, 'name': 'time travel'}, {'id': 9...","[{'cast_id': 4, 'character': 'Lou', 'credit_id...","[{'credit_id': '59ac067c92514107af02c8c8', 'de...",12314651.0,Train
1,2,"[{'id': 107674, 'name': 'The Princess Diaries ...",40000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,tt0368933,en,The Princess Diaries 2: Royal Engagement,Mia Thermopolis is now a college graduate and ...,8.248895,...,113.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,It can take a lifetime to find true love; she'...,The Princess Diaries 2: Royal Engagement,"[{'id': 2505, 'name': 'coronation'}, {'id': 42...","[{'cast_id': 1, 'character': 'Mia Thermopolis'...","[{'credit_id': '52fe43fe9251416c7502563d', 'de...",95149435.0,Train
2,3,NaN,3300000,"[{'id': 18, 'name': 'Drama'}]",http://sonyclassics.com/whiplash/,tt2582802,en,Whiplash,"Under the direction of a ruthless instructor, ...",64.299990,...,105.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The road to greatness can take you to the edge.,Whiplash,"[{'id': 1416, 'name': 'jazz'}, {'id': 1523, 'n...","[{'cast_id': 5, 'character': 'Andrew Neimann',...","[{'credit_id': '54d5356ec3a3683ba0000039', 'de...",13092000.0,Train
3,4,NaN,1200000,"[{'id': 53, 'name': 'Thriller'}, {'id': 18, 'n...",http://kahaanithefilm.com/,tt1821480,hi,Kahaani,Vidya Bagchi (Vidya Balan) arrives in Kolkata ...,3.174936,...,122.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,NaN,Kahaani,"[{'id': 10092, 'name': 'mystery'}, {'id': 1054...","[{'cast_id': 1, 'character': 'Vidya Bagchi', '...","[{'credit_id': '52fe48779251416c9108d6eb', 'de...",16000000.0,Train
4,5,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",NaN,tt1380152,ko,마린보이,Marine Boy is the story of a former national s...,1.148070,...,118.0,"[{'iso_639_1': 'ko', 'name': '한국어/조선말'}]",Released,NaN,Marine Boy,NaN,"[{'cast_id': 3, 'character': 'Chun-soo', 'cred...","[{'credit_id': '52fe464b9251416c75073b43', 'de...",3923970.0,Train


In [6]:
#Get info on new frame total
total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7398 entries, 0 to 7397
Data columns (total 24 columns):
id                       7398 non-null int64
belongs_to_collection    1481 non-null object
budget                   7398 non-null int64
genres                   7375 non-null object
homepage                 2366 non-null object
imdb_id                  7398 non-null object
original_language        7398 non-null object
original_title           7398 non-null object
overview                 7376 non-null object
popularity               7398 non-null float64
poster_path              7396 non-null object
production_companies     6984 non-null object
production_countries     7241 non-null object
release_date             7397 non-null object
runtime                  7392 non-null float64
spoken_languages         7336 non-null object
status                   7396 non-null object
tagline                  5938 non-null object
title                    7395 non-null object
Keywords           

In [7]:
#Fill in null values in crew with 'Missing Crew'
total['crew'] = total['crew'].fillna('Missing Crew')

In [8]:
#Fill the null values for tagline with empty space
total['tagline'] = total['tagline'].fillna(' ')

In [9]:
#Fill the null values for overview with empty space
total['overview'] = total['overview'].fillna(' ')

In [10]:
#Fill the NA values for runtime
total['runtime']= total['runtime'].fillna(total['runtime'].mean())

In [11]:
#Create a function to parse columns
def parse_column(col):
    col = str(col)
    col = col.replace('[','')
    col = col.replace(']','')
    col = col.replace("'",'')
    col = col.replace('{','')
    col = col.replace('}','')
    lst = col.split(',')
    output = []
    for i in lst:
        word = i
        if word[:5] == ' name':
            string = word.split(':')
            string = string[1]
            string = string.lstrip()
            string = string.lower()
            output.append(string)
    return output

In [12]:
#Create function to convert column to list
def list_to_string(col):
    output = ""
    for i in col:
        output = output + i
        output = output + ' '
    return output

In [13]:
#Create function to find the director
def find_director(crew_string):
    if " 'job': 'Director'" in crew_string:
        crew = crew_string
        crew = crew.replace('[', '')
        crew = crew.replace(']', '')
        crew = crew.replace('{', '')
        crew = crew.replace('}', '')
        crew_list = crew.split(',')
        director_index = crew_list.index(" 'job': 'Director'")
        name_index = director_index + 1
        output = crew_list[name_index][10:-1]
    else:
        output = 'Missing Director'
    return output

In [14]:
#Apply custom function to crew new column 'Director'
total['director'] = total['crew'].apply(find_director)

In [15]:
#Create a new column that gives a ratio of movie performance (if revenue and budget not NULL or 0)
train_index = total[total['Dataset'] == 'Train'].index
total['Performance Ratio'] = total['revenue'].loc[train_index] / total['budget'].loc[train_index]

In [16]:
#Create function to replace infinity in Performance Ratio column
import math
def replace_inf(col):
    if col in np.array([math.inf], dtype='float64'):
        col = 0
    else:
        col = col
    return col

In [17]:
#Replace inf in Performance Ratio column
total['Performance Ratio'] = total['Performance Ratio'].apply(replace_inf)

In [18]:
#Check that inf has been replaced as zero
total['Performance Ratio'].value_counts()

0.000000      812
1.000000        8
2.000000        6
1.333333        5
1.500000        3
3.000000        3
4.000000        3
5.000000        3
4.666667        2
5.500000        2
0.150000        2
10.000000       2
100.000000      2
3.400000        2
3.500000        2
2.050000        2
2.500000        2
0.250000        2
0.083333        2
0.200000        2
10.454234       1
5.123191        1
0.163184        1
2.930454        1
1.666756        1
1.344787        1
5.180542        1
1.092134        1
0.019936        1
3.982651        1
             ... 
0.770047        1
3.101309        1
3.035860        1
7.170440        1
8.374117        1
2.384183        1
3.040442        1
2.096971        1
2.378839        1
2.837813        1
0.711111        1
2.278981        1
4.910743        1
0.122080        1
0.085722        1
0.296667        1
5.121016        1
1.429945        1
1.247165        1
36.764313       1
0.285811        1
2.933333        1
3.004000        1
3.689379        1
0.719771  

In [19]:
#Create a column for ratio of director's average performance
train_index = total[total['Dataset'] == 'Train'].index
total_copy = total.loc[train_index].copy()
director_ratio = total_copy[['Performance Ratio', 'director']].groupby(['director']).mean()

dir_ratio_dict = director_ratio.to_dict()['Performance Ratio']
print(dir_ratio_dict)

{'A.J. Edwards': 0.0, 'A.R. Murugadoss': 8.351648351648352, 'Aaron Katz': 6.17, 'Aaron Norris': 0.0, 'Abbas Kiarostami': 0.0, 'Abbas Tyrewala': 4.826086956521739, 'Abderrahmane Sissako': 0.0, 'Abrid Shine': 8.333333333333334, 'Adam Brooks': 0.0, 'Adam Elliot': 2.9232102467661187, 'Adam Green': 0.0105208, 'Adam McKay': 3.5050922761083743, 'Adam Rapp': 0.03250942857142857, 'Adam Resnick': 0.0, 'Adam Rifkin': 0.1540255576923077, 'Adam Shankman': 4.32587505, 'Adam Wingard': 0.0, 'Adolfo Mart√≠nez P√©rez': 0.0, 'Adrian Lyne': 11.442323588709677, 'Adrienne Shelly': 11.089715, 'Agathiyan': 1.0, 'Akira Kurosawa': 0.0, 'Aksinya Gog': 0.0, 'Alain Chabat': 0.7886846801346802, 'Alan Gibson': 0.0, 'Alan J. Pakula': 8.305882352941177, 'Alan Myerson': 4.692678571428571, 'Alan Parker': 0.894238463942959, 'Alan Poul': 2.2136288, 'Alan Shapiro': 0.7865264394829612, 'Alastair Fothergill': 0.0, 'Albert Hughes': 2.047039397321429, 'Albert Pyun': 0.0, 'Alberto Rodr√≠guez': 0.4774254, 'Alejandro Agresti': 2.

In [20]:
#Create a new column in total for Director ratio
total['Director Ratio'] = None
for i in range(len(total)):
    try:
        director = total['director'].loc[i]
        total['Director Ratio'].loc[i] = dir_ratio_dict[director]
    except KeyError:
        total['Director Ratio'].loc[i] = 0

C:\Users\Matt\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [21]:
#LDA Text Analysis - keywords
added_stop_words = {}
stop_words = text.ENGLISH_STOP_WORDS.union(added_stop_words)
vect = CountVectorizer(max_features=10000, max_df=.5, 
                       stop_words=stop_words)
X = vect.fit_transform(total.Keywords.fillna(''))
lda = LatentDirichletAllocation(n_components=5, learning_method="batch",
                                max_iter=25, random_state=0) 
document_topics = lda.fit_transform(X)
topics_frame = pd.DataFrame(document_topics, columns=['keywords_1','keywords_2', 'keywords_3', 'keywords_4', 'keywords_5'])
total = pd.concat([total,topics_frame],axis=1)

In [22]:
total.columns

Index(['id', 'belongs_to_collection', 'budget', 'genres', 'homepage',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'runtime', 'spoken_languages',
       'status', 'tagline', 'title', 'Keywords', 'cast', 'crew', 'revenue',
       'Dataset', 'director', 'Performance Ratio', 'Director Ratio',
       'keywords_1', 'keywords_2', 'keywords_3', 'keywords_4', 'keywords_5'],
      dtype='object')

In [23]:
#Drop all text columns except genre
total.drop(axis=1,columns=['belongs_to_collection','homepage','imdb_id','original_language',
                          'original_title','overview','poster_path',
                          'production_companies','production_countries',
                          'release_date','spoken_languages','status','tagline',
                          'Keywords','cast','crew','director','Performance Ratio','title'], inplace=True)

In [24]:
#Fill the NA values for runtime
total['runtime']= total['runtime'].fillna(total['runtime'].mean())

In [25]:
#Create a new column for list of genre
total['genres_list'] = total['genres'].apply(parse_column)

In [26]:
#Check the head
total['genres_list'].head()

0                            [comedy]
1    [comedy, drama, family, romance]
2                             [drama]
3                   [thriller, drama]
4                  [action, thriller]
Name: genres_list, dtype: object

In [27]:
#Create Dummy Variables for genres
#Create MultiLabelBinarizer
genres_list = []
for i in total['genres_list']:
    genres_list.extend(i)
genres_set = set(genres_list)
genres_list = list(genres_set)
print(len(genres_list))
mlb = MultiLabelBinarizer()
array = mlb.fit_transform(total['genres_list'])
genres_list.sort()
genres_dataframe = pd.DataFrame(data=array,columns=genres_list)

20


In [28]:
#Combine newdataframe to total
total = pd.concat([total, genres_dataframe], axis=1)

In [29]:
#Normalize columns
total['budget'] = scale(total['budget'])
total['popularity'] = scale(total['popularity'])
total['runtime'] = scale(total['runtime'])

In [30]:
#Check columns / drop genres_list, genres
total.drop(axis=1,columns=['genres','genres_list','Dataset'],inplace=True)

In [31]:
#Check columns
total.columns

Index(['id', 'budget', 'popularity', 'runtime', 'revenue', 'Director Ratio',
       'keywords_1', 'keywords_2', 'keywords_3', 'keywords_4', 'keywords_5',
       'action', 'adventure', 'animation', 'comedy', 'crime', 'documentary',
       'drama', 'family', 'fantasy', 'foreign', 'history', 'horror', 'music',
       'mystery', 'romance', 'science fiction', 'thriller', 'tv movie', 'war',
       'western'],
      dtype='object')

In [32]:
#Split the dataset back into train and test
train_index = total[total['revenue'].isnull() == False].index
test_index = total[total['revenue'].isnull() == True].index
train = total.loc[train_index]
test = total.loc[test_index]
test.drop(['revenue'],axis=1,inplace=True)

In [33]:
#Create a model, X, and y
X = train.drop(['revenue','id'], axis=1)
y = train['revenue']

In [34]:
#Create Linear Model
lm = LinearRegression()

In [35]:
#Fit the model and predict
lm.fit(X, y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [36]:
#Find model score - R squared
lm.score(X, y)

0.6313497533677692

In [37]:
#Apply the model to test
X_2 = test.drop(['id',], axis=1)
y_hat_test = lm.predict(X_2)

In [38]:
y_hat_test

array([ 43510445.7823312 , -16084477.41555601,  20142058.66469055, ...,
        63682817.60172062,  55468864.83134452,   7031028.24123189])

In [39]:
#Create csv for submission
revenue_prediction = pd.Series(y_hat_test)
test = test.reset_index()
test.drop(['index'],axis=1,inplace=True)
test['Revenue'] = revenue_prediction
save_frame = test[['id','Revenue']].copy()
save_frame.to_csv('IMBD Movie Revenue Predictions.csv', index=False)

In [40]:
#Get Metrics
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import mean_absolute_error, mean_squared_error

y_hat_train = lm.predict(X)

MAE = mean_absolute_error(y,y_hat_train)
MSE = mean_squared_error(y,y_hat_train)
RMSE = np.sqrt(MSE)

print("MAE = {}".format(MAE))
print("MSE = {}".format(MSE))
print("RMSE = {}".format(RMSE))

MAE = 45639020.698798776
MSE = 6970746959251846.0
RMSE = 83490999.27089055


In [ ]:
#Score from Kaggle
#6.36415